In [1]:
import random
from os import path
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches

from coco import COCO

# import necessary libraries
from PIL import Image
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as T
import torchvision
import torch
import numpy as np
import cv2

device = torch.device("cuda:0")

# get the pretrained model from torchvision.models
# Note: pretrained=True will get the pretrained weights for the model.
# model.eval() to use the model for inference
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

# Class labels from official PyTorch documentation for the pretrained model
# Note that there are some N/A's 
# for complete list check https://tech.amikelive.com/node-718/what-object-categories-labels-are-in-coco-dataset/
# we will use the same list for this notebook
COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]


def get_prediction(img_path, threshold):
  """
  get_prediction
    parameters:
      - img_path - path of the input image
      - threshold - threshold value for prediction score
    method:
      - Image is obtained from the image path
      - the image is converted to image tensor using PyTorch's Transforms
      - image is passed through the model to get the predictions
      - class, box coordinates are obtained, but only prediction score > threshold
        are chosen.
    
  """
  img = Image.open(img_path)
  transform = T.Compose([T.ToTensor()])
  img = transform(img)
  img.to(device)
  pred = model([img])
  pred_class = [COCO_INSTANCE_CATEGORY_NAMES[i] for i in list(pred[0]['labels'].numpy())]
  pred_boxes = [[(i[0], i[1]), (i[2], i[3])] for i in list(pred[0]['boxes'].detach().numpy())]
  pred_score = list(pred[0]['scores'].detach().numpy())

  print('pred class:')
  print(pred_class)
  print('pred_boxes')
  print(pred_boxes)
  print('pred_score')
  print(pred_score)

  pred_t = [pred_score.index(x) for x in pred_score if x>threshold][-1]
  pred_boxes = pred_boxes[:pred_t+1]
  pred_class = pred_class[:pred_t+1]
  pred_score = pred_score[:pred_t+1]

  
    
  return pred_boxes, pred_class, pred_score
  


def object_detection_api(img_path, threshold=0.5, rect_th=3, text_size=3, text_th=3):
  """
  object_detection_api
    parameters:
      - img_path - path of the input image
      - threshold - threshold value for prediction score
      - rect_th - thickness of bounding box
      - text_size - size of the class label text
      - text_th - thichness of the text
    method:
      - prediction is obtained from get_prediction method
      - for each prediction, bounding box is drawn and text is written 
        with opencv
      - the final image is displayed
  """
  boxes, pred_cls = get_prediction(img_path, threshold)
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  for i in range(len(boxes)):
    cv2.rectangle(img, boxes[i][0], boxes[i][1],color=(0, 255, 0), thickness=rect_th)
    cv2.putText(img,pred_cls[i], boxes[i][0], cv2.FONT_HERSHEY_SIMPLEX, text_size, (0,255,0),thickness=text_th)
  plt.figure(figsize=(20,30))
  plt.imshow(img)
  plt.xticks([])
  plt.yticks([])
  plt.show()

# path to the ground truth annotation and the data
annFile = '/home/test/data/nightowls/nightowls_validation.json'
image_directory = '/home/test/data/nightowls/nightowls_validation'

# load the annotations and the list of images
cocoGt = COCO(annFile)
imgIds = sorted(cocoGt.getImgIds())
print('There are %d images in the training set' % len(imgIds))
annotations = cocoGt.getAnnIds()
print('There are %d annotations in the training set' % len(annotations))

import json

# get a random image and its path
# im_id = imgIds[random.randint(0, len(imgIds))] 
im_id = 7027652 # bus with three persons: 7027652
image = cocoGt.loadImgs(ids=im_id)[0]
file_path = path.join(image_directory, image['file_name'])


# get the annotations of the images
anns = cocoGt.getAnnIds([image['id']])

print(image)
print('File path: ' + str(file_path))
print('Number of annotations in the image: ' + str(len(anns)))

# draw the annotations on the image
img=mpimg.imread(file_path)
fig,ax = plt.subplots(1)
ax.imshow(img)

for ann_id in anns:
    ann = cocoGt.loadAnns(ids=ann_id)[0]
    print(ann)
    bbox = ann['bbox']
    rect = patches.Rectangle((bbox[0],bbox[1]),bbox[2],bbox[3],linewidth=2,edgecolor='g',facecolor='none')
    ax.add_patch(rect)

#print
plt.show()

#object_detection_api(file_path, threshold=0.8)
boxes, pred_cls, pred_score = get_prediction(file_path, threshold=0.5)


for i in range(len(boxes)):
    print(boxes[i])
    print(pred_cls[i])
    
    jsonresult = {
      "image_id": im_id,
      "category_id": 1,
      "score": pred_score[i],
      "bbox": []
    }
    jsonresult['bbox'].append(boxes[i][0][0])
    jsonresult['bbox'].append(boxes[i][0][1])
    jsonresult['bbox'].append(boxes[i][1][0] - boxes[i][0][0])
    jsonresult['bbox'].append(boxes[i][1][1] - boxes[i][0][1])
    
    print(jsonresult)
    

loading annotations into memory...
Done (t=0.21s)
creating index...
index created!
There are 51848 images in the training set
There are 17091 annotations in the training set
{'height': 640, 'width': 1024, 'daytime': 'night', 'file_name': '58c58331bc2601370015aa43.png', 'id': 7027652, 'recordings_id': 36.0, 'timestamp': 6307696984}
File path: /home/test/data/nightowls/nightowls_validation/58c58331bc2601370015aa43.png
Number of annotations in the image: 3
{'occluded': False, 'difficult': False, 'bbox': [179, 233, 41, 119], 'id': 7014321, 'category_id': 1, 'image_id': 7027652, 'pose_id': 4, 'tracking_id': 7001032, 'ignore': 0, 'area': 4879, 'truncated': False}
{'occluded': False, 'difficult': False, 'bbox': [60, 223, 27, 90], 'id': 7014322, 'category_id': 1, 'image_id': 7027652, 'pose_id': 2, 'tracking_id': 7001033, 'ignore': 0, 'area': 2430, 'truncated': False}
{'occluded': False, 'difficult': False, 'bbox': [104, 228, 31, 97], 'id': 7014323, 'category_id': 1, 'image_id': 7027652, 'pose_

<Figure size 640x480 with 1 Axes>

pred class:
['person', 'bus', 'person', 'person', 'person', 'handbag', 'handbag', 'backpack', 'handbag', 'handbag', 'person', 'person', 'bus', 'person', 'handbag', 'backpack', 'suitcase', 'backpack', 'backpack', 'bench', 'handbag', 'person', 'truck', 'backpack', 'handbag', 'truck', 'backpack', 'train', 'car', 'person', 'handbag', 'refrigerator', 'person', 'bus', 'refrigerator', 'person', 'person', 'person', 'truck']
pred_boxes
[[(173.7489, 236.49275), (231.31038, 341.77567)], [(213.97305, 156.19266), (495.48972, 319.37366)], [(106.12935, 231.77364), (136.71959, 320.31104)], [(53.104748, 227.83263), (79.708824, 312.02423)], [(140.7585, 236.40726), (154.66072, 267.28928)], [(120.90885, 289.28326), (138.58212, 311.96783)], [(179.22858, 260.0619), (207.45108, 289.26233)], [(120.253075, 247.7241), (134.23001, 269.40765)], [(126.41931, 249.55669), (136.64928, 268.05908)], [(178.40186, 270.61887), (202.45409, 288.50854)], [(65.36184, 232.01225), (82.295876, 298.00974)], [(39.896633, 224.09863

In [4]:
from coco import COCO
from eval_MR_multisetup import COCOeval

# Ground truth
annFile = '/home/test/data/nightowls/nightowls_validation.json'

# Detections
resFile = '../sample-Faster-RCNN-nightowls_validation.json'

## running evaluation
res_file = open("results.txt", "w")
for id_setup in range(0,4):
    cocoGt = COCO(annFile)
    cocoDt = cocoGt.loadRes(resFile)
    imgIds = sorted(cocoGt.getImgIds())
    cocoEval = COCOeval(cocoGt,cocoDt,'bbox')
    cocoEval.params.imgIds  = imgIds
    cocoEval.evaluate(id_setup)
    cocoEval.accumulate()
    cocoEval.summarize(id_setup,res_file)

res_file.close()

loading annotations into memory...
Done (t=0.18s)
creating index...
index created!
Loading and preparing results...
DONE (t=3.35s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=7.60s).
Accumulating evaluation results...
DONE (t=0.47s).
 Average Miss Rate  (MR) @ Reasonable         [ IoU=0.50      | height=[50:10000000000] | visibility=[0.65:10000000000.00] ] = 21.54%
loading annotations into memory...
Done (t=0.17s)
creating index...
index created!
Loading and preparing results...
DONE (t=4.07s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=6.22s).
Accumulating evaluation results...
DONE (t=0.30s).
 Average Miss Rate  (MR) @ Reasonable_small   [ IoU=0.50      | height=[50:75] | visibility=[0.65:10000000000.00] ] = 32.13%
loading annotations into memory...
Done (t=0.17s)
creating index...
index created!
Loading and preparing results...
DONE (t=3.06s)
creating index...
in